In [17]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  # memory_key="history", # history는 code convention으로 굳이 쓰지 않아도 된다.
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}"),
])

def load_memory(input):
  return memory.load_memory_variables({})["history"] # 메모리에 저장되어 있는 내용을 불러옴

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke({"question": question})
  memory.save_context({"input": question}, {"output": result.content}) # 메모리에 저장
  print(result)

In [18]:
invoke_chain("안녕, 내 이름은 빅퍼슨이야")

content='안녕하세요, 빅퍼슨님! 무엇을 도와드릴까요?'


In [19]:
invoke_chain("오늘 날씨는 어떄?")

content='죄송합니다, 제가 알려드릴 수 있는 지역이나 도시를 말씀해주시면 해당 지역의 날씨 정보를 알려드릴 수 있습니다.'


In [20]:
invoke_chain("내 이름이 뭐라구?")

content='당신의 이름은 빅퍼슨님이십니다. 어떻게 도와드릴까요?'
